In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
cd ../follidet/

In [ ]:
import os
import re
import cv2
import glob
import json
import copy
import warnings
import skimage.io
import albumentations as albu

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter
from torch.utils.data import DataLoader
from matplotlib.patches import Rectangle

In [ ]:
from params import *
from training.main import k_fold_training
from model import define_model
from data.dataset import FollicleDataset
from data.transforms import OCT_preprocess
from data.loader import get_collate_fns
from util.plot import plot_sample, plot_predictions
from util.logger import prepare_log_folder, save_config, create_logger

## Data

In [ ]:
df = pd.read_csv(DATA_PATH + METADATA_PATHS[0]).fillna('')

## Model

In [ ]:
class Config:
    selected_model = "efficientdet_d0"
#     selected_model = "yolov5s"
    num_classes = 1
    conf_thresh = 0.5
    iou_thresh = 0.5
    
    bbox_format = "yolo" if "yolo" in selected_model else "pascal_voc"
    
    # Loss (YOLO)
    gr = 1.0
    autobalance = False
    label_smoothing = 0
    box = 0.05  # box loss gain
    cls = 0.5  # cls loss gain
    cls_pw = 1.0  # cls BCELoss positive_weight
    obj = 1.0  # obj loss gain (scale with pixels)
    obj_pw = 1.0  # obj BCELoss positive_weight
    iou_t = 0.20  # IoU training threshold
    anchor_t = 4.0  # anchor-multiple threshold
    # anchors: 3  # anchors per output layer (0 to ignore)
    fl_gamma = 0.0  # focal loss gamma (efficientDet default gamma=1.5)

In [ ]:
transforms = OCT_preprocess(augment=False, visualize=True, bbox_format=Config.bbox_format)

dataset = FollicleDataset(
    df,
    root="../data/processed/",
    transforms=transforms,
    bbox_format=Config.bbox_format,
    mosaic_proba=1,
)

img, boxes, shapes = dataset[423]
plot_sample(img.numpy().transpose(1, 2, 0), boxes.copy(), bbox_format=Config.bbox_format)

### Model

#### Train

In [ ]:
transforms = OCT_preprocess(augment=True, visualize=False, bbox_format=Config.bbox_format)

dataset = FollicleDataset(
    df,
    root="../data/processed/",
    transforms=transforms,
    bbox_format=Config.bbox_format,
)
img, boxes, shapes = dataset[0]

collate_fn_train, collate_fn_val = get_collate_fns(Config.selected_model)
loader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=False,
    drop_last=True,
    num_workers=4,
#     pin_memory=True,
    collate_fn=collate_fn_train
)

In [ ]:
model = define_model(Config).train().cuda()

In [ ]:
for batch in tqdm(loader):
    imgs, labels, shapes = batch
    nb, _, height, width = imgs.shape

    y = model(imgs.cuda())
    
    loss = model.compute_loss(y, labels)

    print(loss)
    break

#### Val

In [ ]:
transforms = OCT_preprocess(augment=True, visualize=False, bbox_format=Config.bbox_format)

dataset = FollicleDataset(
    df,
    root="../data/processed/",
    transforms=transforms,
    bbox_format=Config.bbox_format,
)
img, boxes, shapes = dataset[0]

collate_fn_train, collate_fn_val = get_collate_fns(Config.selected_model)
loader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=False,
    drop_last=True,
    num_workers=1,
    pin_memory=True,
    collate_fn=collate_fn_val
)

In [ ]:
model = define_model(Config).eval().cuda()

In [ ]:
for batch in tqdm(loader):
    imgs, labels, shapes = batch
    nb, _, height, width = imgs.shape

    y = model(imgs.cuda())
    break

In [ ]:
y

## Main

In [ ]:
BATCH_SIZES = {
    "yolov5s": 32,
    "efficientdet_d0": 16,
    "efficientdet_d1": 16,
}

In [ ]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    img_folder = "../data/processed/"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    save_weights = True
    first_epoch_eval = 3
    verbose_plot = 0
    
    # k-fold
    k = 5
    selected_folds = [0]
    
    # Model
    selected_model = "yolov5s"
#     selected_model = "efficientdet_d1"
    bbox_format = "yolo" if "yolo" in selected_model else "pascal_voc"
    pred_format = "pascal_voc" #if "yolo" in selected_model else "coco"
    num_classes = 1
    
    # Loss (YOLO)
    gr = 1.0
    autobalance = False
    label_smoothing = 0
    box = 0.05  # box loss gain
    cls = 0.5  # cls loss gain
    cls_pw = 1.0  # cls BCELoss positive_weight
    obj = 1.0  # obj loss gain (scale with pixels)
    obj_pw = 1.0  # obj BCELoss positive_weight
    iou_t = 0.20  # IoU training threshold
    anchor_t = 4.0  # anchor-multiple threshold
    # anchors: 3  # anchors per output layer (0 to ignore)
    fl_gamma = 0.0  # focal loss gamma (efficientDet default gamma=1.5)
    
    # NMS
    conf_thresh = 0.5  # 0.1
    iou_thresh = 0.5

    # Training 
    optimizer = "Adam"
    momentum = 0.937
    weight_decay = 0.0005
    
    batch_size = BATCH_SIZES[selected_model]
    val_bs = batch_size
    
    epochs = 10
    lr = 5e-4
    warmup_prop = 0.1
    
#     mix = "cutmix"
#     mix_proba = 0.5
#     mix_alpha = 5
    mosaic_proba = 0


In [ ]:
DEBUG = False
log_folder = None

In [ ]:
%matplotlib inline

if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f"Logging results to {log_folder}")
    save_config(Config, log_folder + "config.json")
    df.to_csv(log_folder + "data.csv", index=False)
    create_logger(directory=log_folder, name="logs.txt")

meters = k_fold_training(Config, df, log_folder=log_folder);

In [ ]:
meter = copy.deepcopy(meters[0])
meter.compute(iou_threshold=0.2)

In [ ]:
meter.plot(10)